In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Unit Test

- corpus maker

In [8]:
from corpusmaker import *

# Test the IMDB corpus
IMDB_corpus: TextCorpus = IMDB(50)
# Use `raw` arrtibute to access the raw text list
print(IMDB_corpus.raw[0])
# Use `random_split` method to split the corpus into train and test
train, test = IMDB_corpus.random_split(test_rate=0.3)

print(f"Datapoints in train: {len(train)}")
print(f"Datapoints in test: {len(test)}")

SPOILERS (ALTHOUGH NONE THAT AREN'T REVEALED IN THE FIRST TWO MINUTES OF THE MOVIE)<br /><br />Robin Williams is actually quite good in this as the friendly, lonely, emotionally stunted loser Sy. He makes a very human, even sympathetic psycho, and really disappears into the character--no small feat for such a recognizable performer. <br /><br />Too bad the rest of the movie is such a waste. The supporting performances (and performers) wouldn't look out of place in a soft-core porno (it doesn't help that every character but Sy is made of 100% cardboard). At times, the director actually seems to be trying to frustrate suspense: we know from the very first moments a) that Sy is a complete whack-job, b) that he survives, and c) that he gets nabbed by the cops at the end. So all we're left to ponder is the hows and the whys, and the answers provided aren't all that interesting.<br /><br />The plot is plodding and contrived, and features some nonsensical moments (for instance, the husband be

- model_subject

In [9]:
from models.model_subject import ModelSubject_Toy
from models.configurations import TaskConfig

cfg = TaskConfig(
    subject_model_name='Toy',
    input_corpus_dataset_name='Pile',
    input_corpus_datapoint_num=50,
    context_window=2,
    max_bytes_per_file=2000,
)

# Test the toy model
toy_model = ModelSubject_Toy(cfg)
print(f"- The Toy Model has {toy_model.n_layer} layers")
print(f"- The Toy Model embedding has {toy_model.n_embed} dimensions")
print(f"- The Toy Model has context window of {toy_model.context_window} words")

inference_result = toy_model.inference('this sentence was not used for training')
print("\n         ", end="")
for i in range(toy_model.n_embed):
    print(f"Dimension {i} ", end="")
print("Layer       TokenIdx")
print(inference_result, '\n')
print(inference_result.shape)

- The Toy Model has 3 layers
- The Toy Model embedding has 4 dimensions
- The Toy Model has context window of 2 words

         Dimension 0 Dimension 1 Dimension 2 Dimension 3 Layer       TokenIdx
tensor([[1.0198e-01, 3.3222e-01, 4.2252e-01, 2.0119e-01, 0.0000e+00, 8.8800e+02],
        [7.1443e-01, 1.2758e-01, 3.0770e-01, 7.1691e-01, 1.0000e+00, 8.8800e+02],
        [7.1154e-01, 5.8066e-01, 8.7207e-01, 1.5621e-01, 2.0000e+00, 8.8800e+02],
        [2.4452e-01, 5.8278e-02, 2.1006e-01, 7.3296e-01, 0.0000e+00, 9.9900e+02],
        [3.1322e-01, 8.0973e-01, 7.8859e-01, 2.9062e-02, 1.0000e+00, 9.9900e+02],
        [2.3914e-01, 2.8997e-02, 2.6225e-01, 2.8272e-01, 2.0000e+00, 9.9900e+02]],
       device='cuda:0') 

torch.Size([6, 6])


- embeddings

In [10]:
from pipeline import make_embeddings_from_config

try:
    make_embeddings_from_config(cfg)
except FileExistsError as e:
    print(e)

./dataset/Toy_Pile_50_2 already exists


In [11]:
from embeddings import LoadEmbeddingDataset

dataset = LoadEmbeddingDataset(cfg)

for i in range(dataset.file_num):
    file = dataset[i]
    print(i, file.shape, file.device)

Checking ./dataset/Toy_Pile_50_2...
file_num: 4, tensor_num_per_file: 76


Checking files: 100%|██████████| 4/4 [00:00<00:00, 41120.63it/s]

./dataset/Toy_Pile_50_2 is complete
0 torch.Size([76, 6]) cpu
1 torch.Size([76, 6]) cpu
2 torch.Size([76, 6]) cpu
3 torch.Size([72, 6]) cpu


- classifier

In [12]:
from classifier import *
from models.configurations import *
from pipeline import pipeline

task_config = TaskConfig(
    subject_model_name='meta-llama/Llama-2-13b-chat-hf',
    input_corpus_dataset_name='Pile',
    input_corpus_datapoint_num=3000,
    context_window=30,
    # normalized_every_vector=True,
    # append_msg="normalized",
)

train_config = TrainConfig(
    sparse_penalty=0,
)

pipeline(
    task_cfg=task_config,
    train_cfg=train_config,
)

Building dataset meta-llama-Llama-2-13b-chat-hf_Pile_3000_30...


KeyboardInterrupt: 